# Selecionar Features para um modelo

## Tipos de métodos

- Filter Methods: utilização medidas estatísticas para atribuir um score para cada feature. Normalmente são utilizados **testes univariados** que considera o qual independente é a feature com a variável alvo.

**EXEMPLO:** *scores com coeficiente de correlação*, *chi square*

- Wrapper Methods: um conjunto de features são selecionadas e combinadas, em seguida são preparadas, avaliadas e comparadas. Para avaliar a combinação é utilizado um modelo preditivo e em seguido atribuir um score baseado em uma acurácia.

**EXEMPLO:** *algoritmo RFE*

- Embedded Methods: aprendem quais features tem melhor contribuição para a acurácia do modelo no momento de construção do modelo.

**EXEMPLO:** *algoritmos Lasso, método de penalização, Ridge Regression**

## Exemplo de seleção de features

In [1]:
import pandas as pd

In [2]:
col_names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']

df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv', names=col_names)

In [3]:
df.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   preg    768 non-null    int64  
 1   plas    768 non-null    int64  
 2   pres    768 non-null    int64  
 3   skin    768 non-null    int64  
 4   test    768 non-null    int64  
 5   mass    768 non-null    float64
 6   pedi    768 non-null    float64
 7   age     768 non-null    int64  
 8   class   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
X = df.drop(['class'], axis=1)
y = df['class']

## Testes estatísticos

Usados para selecionar atributos que possuem forte relação com a variável alvo, ou seja, a que estamos tentando prever.

- **f_classif**: *é adequado quando os dados são numéricos e a variável alvo é categórica*
- **f_refression**: *indicado para problemas de regressão*
- **mutual_info_class**: *indicado para casos onde há uma dependência linear entre as features e o target*

**Scikit-learn**: Podemos utilizar a função *SelectkBest* que permite o uso de diversos testes estatísticos. Basta informar para a função o número **k** de features a serem selecionadas como sendo as mais importantes.

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

In [7]:
_f_classif = SelectKBest(score_func=f_classif, k=4)
fit = _f_classif.fit(X, y)
features = fit.transform(X)

In [8]:
# Features com maior pontuação
cols = fit.get_support(indices=True)
df.iloc[:,cols].head(10)

,preg,plas,mass,age
0,6,148,33.6,50
1,1,85,26.6,31
2,8,183,23.3,32
3,1,89,28.1,21
4,0,137,43.1,33
5,5,116,25.6,30
6,3,78,31.0,26
7,10,115,35.3,29
8,2,197,30.5,53
9,8,125,0.0,54


## Chi2

In [9]:
from sklearn.feature_selection import chi2

In [10]:
test = SelectKBest(chi2, k=4)
fit = test.fit(X, y)
features = fit.transform(X)

In [12]:
print(features)

[[148.    0.   33.6  50. ]
 [ 85.    0.   26.6  31. ]
 [183.    0.   23.3  32. ]
 ...
 [121.  112.   26.2  30. ]
 [126.    0.   30.1  47. ]
 [ 93.    0.   30.4  23. ]]


In [14]:
fit.get_support(indices=True)
cols = fit.get_support(indices=True)
df.iloc[:,cols].head()

,plas,test,mass,age
0,148,0,33.6,50
1,85,0,26.6,31
2,183,0,23.3,32
3,89,94,28.1,21
4,137,168,43.1,33


## RFE - Recursive Feature Elimination
----

Remove de forma recursiva as features e constrói o modelo com as features que restarem.

- Para identificar as features ou combinação de features que trazem melhor resultado para o modelo é utilizada a acurácia.
- É necessário um modelo para aplicação dessa técnica
- Em bases grandes o tempo de processamento pode ser significativo

*Se não informar ao RFE a quantidade de features, o mesmo tentará reduzir pela metade esse valor

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

In [18]:
model = LogisticRegression(max_iter=2000)
rfe = RFE(model, 4)
fit = rfe.fit(X, y)

In [19]:
print(f'Nº de features {fit.n_features_}')

Nº de features 4


In [21]:
cols = fit.get_support(indices=True)
df.iloc[:,cols].head()

,preg,plas,mass,pedi
0,6,148,33.6,0.627
1,1,85,26.6,0.351
2,8,183,23.3,0.672
3,1,89,28.1,0.167
4,0,137,43.1,2.288
